In [1]:
import mysql.connector as sql

In [2]:
class Models:
    def __init__(self, query):
        self.query = query
        self.conn = sql.connect(
                        host = "localhost",
                        user="root",
                        passwd = "",
                        database = "db_ta_1811501327"
                    )
        self.curr = self.conn.cursor()
        
    def select(self):
        self.curr.execute(self.query)
        row_headers = [x[0] for x in self.curr.description]
        result = self.curr.fetchall()
        
        self.curr.close()
        self.conn.close()
        
        json_data = []
        for data in result:
            json_data.append(dict(zip(row_headers, data)))
            
        return json_data
    
    def query_sql(self, values):
        self.curr.execute(self.query, values)
        self.conn.commit()
        self.curr.close()
    
    def query_sql_multiple(self, values):
        self.curr.executemany(self.query, values)
        self.conn.commit()
        self.curr.close()
        
    def query_deleteAll(self):
        self.curr.execute(self.query)
        self.conn.commit()

In [3]:
instance_model = Models('SELECT id, clean_text FROM tbl_tweet_preprocessing WHERE label IS NULL')
data_no_label = instance_model.select()

instance_model = Models('SELECT positive_word FROM tbl_lexicon_positive')
lexicon_positif = instance_model.select()

instance_model = Models('SELECT negative_word FROM tbl_lexicon_negative')
lexicon_negatif = instance_model.select()

In [4]:
teks_data = []
skor_data = []
total_positif = []
total_negatif = []
jumlah_netral = 0

In [5]:
data_ubah = []

print('\n-- PROSES '+ str(len(data_no_label)) +' DATA --')

for index, data in enumerate(data_no_label):
    skor = 0
    jumlah_positif = 0
    jumlah_negatif = 0
    # Menghitung jumlah skor pada teks bersih dengan kamus
    for clean_text in data['clean_text'].split(): # Tokenizing
        for data_p in lexicon_positif:	# loop data kata positif
            if clean_text == data_p['positive_word']:
                skor += 1
                jumlah_positif += 1
                break

        for data_n in lexicon_negatif:	# loop data kata negatif
            if clean_text == data_n['negative_word']:
                skor -= 1
                jumlah_negatif += 1
                break

    if skor > 0:
        label = 'positif'
    elif skor < 0:
        label = 'negatif'
    else:
        label = 'netral'
        jumlah_netral += 1

    try:
        data_ubah.append((label, data['id']))
    except:
        print('\nGagal mengubah data'+ str(data_ubah['id']) +'\n')
    print(index+1)
    
    instance_model = Models('UPDATE tbl_tweet_preprocessing SET label=%s WHERE id=%s')
    instance_model.query_sql_multiple(data_ubah)

print('\nSELESAI\n')



-- PROSES 0 DATA --

SELESAI

